<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Cesar Fukushima Kim Bresciani, 21004780' #@param {type:"string"}

In [ ]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
proprietarios_imoveis = df.groupby('host_id').id.count().sort_values(ascending=False)
proprietarios_imoveis.head()

host_id
219517861    7
30283594     4
120762452    4
19303369     4
1513294      2
Name: id, dtype: int64

In [ ]:
print(df[df.host_id == 219517861].host_name)
print(df[df.host_id == 30283594].host_name)
print(df[df.host_id == 120762452].host_name)

676    Sonder (NYC)
677    Sonder (NYC)
678    Sonder (NYC)
707    Sonder (NYC)
723    Sonder (NYC)
727    Sonder (NYC)
747    Sonder (NYC)
Name: host_name, dtype: object
241    Kara
242    Kara
317    Kara
691    Kara
Name: host_name, dtype: object
402    Stanley
419    Stanley
438    Stanley
444    Stanley
Name: host_name, dtype: object


# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [ ]:
anunciantes = df.groupby('host_id').id.count()

anunciantes_mais_de_um = anunciantes[anunciantes > 1]

anunciantes_um = anunciantes[anunciantes == 1]

preco_mais_de_um_imovel = df[df.host_id.isin(anunciantes_mais_de_um.index)]['price'].mean()
preco_mais_um_imovel = df[df.host_id.isin(anunciantes_um.index)]['price'].mean()

In [ ]:
...

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
df_manhattan = df[df.neighbourhood_group == 'Manhattan']

Q1_manhattan = df_manhattan['price'].quantile(0.25)
Q3_manhattan = df_manhattan['price'].quantile(0.75)
IQR_manhattan = Q3_manhattan - Q1_manhattan

lower_bound_manhattan = Q1_manhattan - 1.5 * IQR_manhattan
upper_bound_manhattan = Q3_manhattan + 1.5 * IQR_manhattan

outliers_manhattan = df_manhattan[(df_manhattan['price'] < lower_bound_manhattan) | (df_manhattan['price'] > upper_bound_manhattan)]

perc_Manhattan = (len(outliers_manhattan) / len(df_manhattan)) * 100

In [ ]:
...

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [ ]:
# ordem beautiful,sunny,spacious,heart

df.name.str.contains('beautiful', case=False)

df_nome = df

for i in range(len(df)):
  if df_nome.name.str.contains('beautiful', case=False)[i]:
    df_nome.loc[i, 'word'] = 'beautiful'
  elif df_nome.name.str.contains('sunny', case=False)[i]:
    df_nome.loc[i, 'word'] = 'sunny'
  elif df_nome.name.str.contains('spacious', case=False)[i]:
    df_nome.loc[i, 'word'] = 'spacious'
  elif df_nome.name.str.contains('heart', case=False)[i]:
    df_nome.loc[i, 'word'] = 'heart'
  else:
    df_nome.loc[i, 'word'] = 'others'

#df_nome.tail()

In [ ]:
df.groupby('word').price.mean().sort_values(ascending=False)


word
heart        166.388889
others       157.658106
beautiful    141.785714
sunny        137.674419
spacious     128.516667
Name: price, dtype: float64

In [ ]:
preco_heart = df[df.word == 'heart'].price.mean()
preco_spacious = df[df.word == 'spacious'].price.mean()
print(f'Preço médio de heart: {preco_heart:.2f}')
print(f'Preço médio de spacious: {preco_spacious:.2f}')

Preço médio de heart: 166.39
Preço médio de spacious: 128.52


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [ ]:
df_merge = pd.merge(df_taxes, df)
df_merge.head()

id   type   value                                              name  \
0  35907127  taxes  1315.0                       Huge 1 Bedroom Apt +balcony   
1    804815  taxes     NaN                       Washington Heights homestay   
2  22328585  taxes   900.0                                 Home sweet Harlem   
3  27876600  taxes   869.0       Peaceful garden + private studio in the UES   
4  34976037  taxes  1629.0  Stunning Studio in Lux Building in the UWS #6115   

     host_id host_name neighbourhood_group       neighbourhood  latitude  \
0  265498730   Tatiana           Manhattan      Hell's Kitchen  40.76212   
1    4112902   Tiffany           Manhattan  Washington Heights  40.83329   
2  154494056   Scarlet           Manhattan              Harlem  40.81991   
3   43888084     Elena           Manhattan     Upper East Side  40.78057   
4  116305897     Laura           Manhattan     Upper West Side  40.79058   

   longitude        room_type  price  minimum_nights  number_of_reviews  \
0  -73.99771  Entire home/apt    280               3                  2   
1  -73.93834     Private room     36               1                 33   
2  -73.95248     Private room     75               1                  4   
3  -73.95186  Entire home/apt    248               4                  3   
4  -73.97246  Entire home/apt    150              30                  0   

  last_review  reviews_per_month  calculated_host_listings_count  \
0  2019-07-01               2.00                               1   
1  2019-06-18               0.77                               1   
2  2018-05-14               0.22                               1   
3  2019-04-16               0.31                               1   
4         NaN                NaN                               9   

   availability_365    word  
0               341  others  
1               211  others  
2                 0  others  
3               100  others  
4               266  others

In [ ]:
df_test = df_merge
for i in range(len(df_test)):
  if df_test.value.isnull()[i]:
    df_test.loc[i, 'value'] = 0
df_test.head()

id   type   value                                              name  \
0  35907127  taxes  1315.0                       Huge 1 Bedroom Apt +balcony   
1    804815  taxes     0.0                       Washington Heights homestay   
2  22328585  taxes   900.0                                 Home sweet Harlem   
3  27876600  taxes   869.0       Peaceful garden + private studio in the UES   
4  34976037  taxes  1629.0  Stunning Studio in Lux Building in the UWS #6115   

     host_id host_name neighbourhood_group       neighbourhood  latitude  \
0  265498730   Tatiana           Manhattan      Hell's Kitchen  40.76212   
1    4112902   Tiffany           Manhattan  Washington Heights  40.83329   
2  154494056   Scarlet           Manhattan              Harlem  40.81991   
3   43888084     Elena           Manhattan     Upper East Side  40.78057   
4  116305897     Laura           Manhattan     Upper West Side  40.79058   

   longitude        room_type  price  minimum_nights  number_of_reviews  \
0  -73.99771  Entire home/apt    280               3                  2   
1  -73.93834     Private room     36               1                 33   
2  -73.95248     Private room     75               1                  4   
3  -73.95186  Entire home/apt    248               4                  3   
4  -73.97246  Entire home/apt    150              30                  0   

  last_review  reviews_per_month  calculated_host_listings_count  \
0  2019-07-01               2.00                               1   
1  2019-06-18               0.77                               1   
2  2018-05-14               0.22                               1   
3  2019-04-16               0.31                               1   
4         NaN                NaN                               9   

   availability_365    word  
0               341  others  
1               211  others  
2                 0  others  
3               100  others  
4               266  others

In [ ]:
# Manhattan preco medio taxes
df_test[(df_test.neighbourhood_group == 'Manhattan') & (df_test.type == 'taxes')].value.mean().astype(int)

np.int64(888)

In [ ]:
df_test[df_test.type == 'taxes'].groupby(df_test.neighbourhood_group).value.mean().astype(int)

neighbourhood_group
Bronx             851
Brooklyn          858
Manhattan         888
Queens            804
Staten Island    1157
Name: value, dtype: int64

In [ ]:
df_test[(df_test.neighbourhood_group == 'Manhattan') & (df_test.type == 'fines')].value.mean().astype(int)

np.int64(314)

In [ ]:
df_test[df_test.type == 'fines'].groupby(df_test.neighbourhood_group).value.mean().astype(int)

neighbourhood_group
Bronx            126
Brooklyn         314
Manhattan        314
Queens           322
Staten Island    581
Name: value, dtype: int64

# Nova seção

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
df_report = pd.DataFrame(columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id', 'line_id'])
df_report_base = df_test[['neighbourhood_group', 'neighbourhood', 'price']]
df_report_base.sort_values(by=['neighbourhood_group', 'neighbourhood'], inplace=True)
df_report_base.tail()

<ipython-input-148-1e7c5e0bd185>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report_base.sort_values(by=['neighbourhood_group', 'neighbourhood'], inplace=True)


neighbourhood_group neighbourhood  price
1476       Staten Island   South Beach    100
437        Staten Island    St. George    130
1201       Staten Island    St. George    130
384        Staten Island     Todt Hill    135
1148       Staten Island     Todt Hill    135

In [ ]:
#Fazendo paginas e linhas

pagina_atual = 1
linha_atual = 1
for i in range(len(df_report_base)):
  # Se troca neighbourhood_group, gerar media de price do neighbourhood_group em nova linha
  if df_report_base.neighbourhood_group.iloc[i] != df_report_base.neighbourhood_group.iloc[i-1]:
    pagina_atual += 1
    linha_atual = 1
  # Cada pagina contem 30 linhas
  if linha_atual > 30:
    pagina_atual += 1
    linha_atual = 1
  df_report.loc[i] = [df_report_base.neighbourhood_group.iloc[i], df_report_base.neighbourhood.iloc[i], df_report_base.price.iloc[i], pagina_atual, linha_atual]
  linha_atual += 1

In [ ]:
df_report

neighbourhood_group      neighbourhood  price  page_id  line_id
0                  Bronx            Belmont     29        2        1
1                  Bronx            Belmont     29        2        2
2                  Bronx  Claremont Village     67        2        3
3                  Bronx  Claremont Village     67        2        4
4                  Bronx          Concourse     40        2        5
...                  ...                ...    ...      ...      ...
1523       Staten Island        South Beach    100       55        8
1524       Staten Island         St. George    130       55        9
1525       Staten Island         St. George    130       55       10
1526       Staten Island          Todt Hill    135       55       11
1527       Staten Island          Todt Hill    135       55       12

[1528 rows x 5 columns]

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 14634


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7